In [3]:
import imaplib
import pprint
import json
import email

imap_host = 'imap.gmail.com'

# connect to host using SSL

imap = imaplib.IMAP4_SSL(imap_host)

def get_auth_info():
    filename = "auth.json"
    
    auth_data = json.load(open(filename))
    email = auth_data["email"]
    password = auth_data["password"]
   
    return email, password

def login_to_email(username, password):
    
    # login to server
    imap.login(username, password)
    
    imap.select('Inbox')
    
def search_inbox(imap, phrase):
    search_str = f'(FROM "{phrase}")'
    tmp, data = imap.search(None, search_str)
    result_list = data[0].split()
    
    return result_list

def fetch_email(email_id):
    tmp, email_data = imap.fetch(email_id, '(RFC822)')
    parsed_email = email.message_from_bytes(email_data[0][1])
    
    return parsed_email

def write_file_to_hd(filename, filedata):
    file_object = open(filename, "wb")
    file_object.write(filedata)
    file_object.close()
    
def download_attachment(parsed_email):
    for part in parsed_email.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get("Content-Disposition") is None:
            continue
        filename = part.get_filename()
        filedata = part.get_payload(decode = True)
        
        write_file_to_hd(filename + ".pdf", filedata)
        print(f"Downloaded {filename}.pdf")
        

#Get the auth info from the auth.json file
username, password = get_auth_info()

# Use the username and password to login to gmail account using imap
login_to_email(username, password)

# Search for emails from Freshmenu
result_list = search_inbox(imap, "FreshMenu")

# Take the 4th last email from search result
# email_id = result_list[-4]

for email_id in result_list:
    # Convert Raw email data to parsed email
    parsed_email = fetch_email(email_id)
    
    # Download the attachment if any
    download_attachment(parsed_email)

Downloaded 10720938_FM-KA-2287171.pdf
Downloaded emailer_34-2.jpg.pdf
Downloaded emailer_34-1.jpg.pdf
Downloaded emailer_33.jpg.pdf
Downloaded emailer_34.jpg.pdf
Downloaded emailer_33.png.pdf
Downloaded emailer_32.png.pdf
Downloaded emailer_04.jpg.pdf
Downloaded emailer_11.jpg.pdf
Downloaded emailer_18.jpg.pdf
Downloaded emailer_31.jpg.pdf
Downloaded logo.png.pdf
Downloaded 28081149_FM-KA-5490081.pdf
